In [ ]:
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5 ddgs wikipedia

In [ ]:
!pip install langchain-community
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
import math

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GEM_API_KEY")  # paste your key
print(" API key set.")

 API key set.


In [ ]:
# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    convert_system_message_to_human=True
)

In [ ]:
# Define Calculator Tool
def calculator(expression):
    """
    A simple calculator that can evaluate mathematical expressions.
    Use this for any math calculations.
    """
    try:
        # Clean the expression
        expression = expression.replace("^", "**")  # Handle exponentiation
        # Safe evaluation of mathematical expressions
        allowed_names = {
            k: v for k, v in math.__dict__.items() if not k.startswith("__")
        }
        allowed_names.update({"abs": abs, "round": round, "min": min, "max": max})

        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"


In [ ]:
# Initialize search tools

search = DuckDuckGoSearchRun()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())


In [ ]:
# Create tools list
tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Useful for mathematical calculations. Input should be a mathematical expression like '2+2' or 'sqrt(16)' or '10*5+3'"
    ),
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Useful for searching current information on the internet. Use this when you need recent information or facts."
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="Useful for getting detailed information about people, places, events, concepts from Wikipedia."
    )
]

In [ ]:
# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

/tmp/ipython-input-1818075595.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
# Demo function to run examples
def run_demo():
    """Run demonstration examples"""

    print(" Multi-Tool Agent Demo")
    print("=" * 50)

    examples = [
        "What is 15 * 23 + 45?",
        "Who is the current president of France and what is the square root of 144?",
        "Search for the latest news about Agentic AI developments in 2025",
        "Tell me about Albert Einstein and calculate 299792458 / 1000000",
        "What is the weather like today in Hyderabad, India and what is 2^10?"
    ]

    for i, question in enumerate(examples, 1):
        print(f"\n Example {i}: {question}")
        print("-" * 40)
        try:
            response = agent.run(question)
            print(f" Response: {response}")
        except Exception as e:
            print(f" Error: {str(e)}")
        print("\n" + "="*50)


In [ ]:
# Interactive function
def chat_with_agent():
    """Interactive chat with the agent"""
    print("\n Interactive Chat Mode")
    print("Type 'quit' to exit")
    print("-" * 30)

    while True:
        user_input = input("\n You: ").strip()

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print(" Goodbye!")
            break

        if user_input:
            try:
                response = agent.run(user_input)
                print(f" Agent: {response}")
            except Exception as e:
                print(f" Error: {str(e)}")


In [ ]:
# Main execution
if __name__ == "__main__":
    print(" Setting up Multi-Tool Agent...")

    # Check if API key is set
    if os.environ.get("GOOGLE_API_KEY") == "your-gemini-api-key-here":
        print("  Please set your actual Gemini API key!")
        print("Replace 'your-gemini-api-key-here' with your actual API key")
    else:
        print(" API key configured")

        # Run demo
        choice = input("\nRun demo examples? (y/n): ").lower().strip()
        if choice == 'y':
            run_demo()

        # Start interactive mode
        choice = input("\nStart interactive chat? (y/n): ").lower().strip()
        if choice == 'y':
            chat_with_agent()


 Setting up Multi-Tool Agent...
 API key configured

Run demo examples? (y/n): y
 Multi-Tool Agent Demo

 Example 1: What is 15 * 23 + 45?
----------------------------------------


> Entering new AgentExecutor chain...


/tmp/ipython-input-3744425326.py:20: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(question)
/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I need to calculate the value of the expression 15 * 23 + 45. I can use the calculator tool for this.
Action: Calculator
Action Input: 15 * 23 + 45
Observation: The result is: 390
Thought:

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I have calculated the value of the expression.
Final Answer: 390

> Finished chain.
 Response: 390


 Example 2: Who is the current president of France and what is the square root of 144?
----------------------------------------


> Entering new AgentExecutor chain...


...

The search results provide multiple sources for the weather in Hyderabad, India, but don't give a specific current temperature or conditions. I will try a more direct search.
Action: DuckDuckGo Search
Action Input: "current weather in Hyderabad India"
Observation: 2 days ago - Hyderabad weather today shows overcast skies with temperatures ranging from 20°C minimum to 27°C maximum on Friday, September 26, 2025. Weather in Hyderabad today features generally cloudy conditions with moderate rain continuing throughout the day. Current humidity remains high at 87-100% ... Toggle navigation. Hyderabad weather powered by DoINeedAJacket.com. °C. No. you don't need a jacket based on the weather in Hyderabad , Telangana, India 24 minutes ago. Stay updated with the latest weather conditions in India . Get real-time information on temperature, humidity, and overall climate to plan your day effectively. Currently in Moscow. 00:21 - 25 September, Thursday. +6°C.Variable cloud, no precipitation. Detai

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


The current weather in Hyderabad, India is overcast skies with temperatures ranging from 20°C minimum to 27°C maximum. The humidity is high at 87-100%. Now I need to calculate 2^10.
Action: Calculator
Action Input: 2^10
Observation: The result is: 1024
Thought:

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I now know the final answer
Final Answer: The weather in Hyderabad, India today is overcast skies with temperatures ranging from 20°C to 27°C and high humidity (87-100%). 2^10 is 1024.

> Finished chain.
 Response: The weather in Hyderabad, India today is overcast skies with temperatures ranging from 20°C to 27°C and high humidity (87-100%). 2^10 is 1024.


Start interactive chat? (y/n): n
